In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


print(plt.style.available)
%matplotlib inline
driver = webdriver.Edge()
base_url = "https://cq.zu.fang.com/house-a058-b010742/"


['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [2]:

start_page = 1
end_page = 9  # 总页数

# 用于存储爬取的 HTML 内容
string_list = []

# 遍历所有页面
for page in range(start_page, end_page + 1):
    try:
        # 构造当前页面的 URL
        if page == 1:
            current_url = base_url
        else:
            current_url = f"{base_url}i3{page}/"
        
        # 打开当前页面
        driver.get(current_url)
        
        table = driver.find_element('class name', 'houseList')
        string_list.append(table.get_attribute('outerHTML'))  # Collect raw HTML
        
    except NoSuchElementException as e:
        print(f"Error on page {page}: {e}")
        break  # 如果出现错误，退出循环


In [3]:
string_list

['<div class="houseList"><dl class="list hiddenMap rel"><dt class="img rel floatl" id="rentid_D09_1_01" data-type="houselistshow" data-ajax="{&quot;HouseId&quot;:&quot;205056212&quot;,&quot;AgentID&quot;:&quot;166130002&quot;,&quot;HouseType&quot;:&quot;juhe&quot;,&quot;ListingType&quot;:&quot;1,2&quot;}"><a target="_blank" href="/chuzu/3_205056212_1.htm" data_channel="1,2" source_page="1"><img class="b-lazy lazyload" usertype="1ccc" data-original="//cdnsfb.soufunimg.com/viewimage/sfb/houseverfiy/2025_8/25/M14/19/3157202d-b5e0-47ef-9279-f21a39d50a7c/275x207.jpg?t=1" src="//img.soufunimg.com/secondhouse/image/loading100_75.gif" onerror="imgiserror(this,\'//cdnsfb.soufunimg.com/sfb/houseverfiy/2025_8/25/M14/19/3157202d-b5e0-47ef-9279-f21a39d50a7c.jpg\')"></a><p class="icon_box"><span class="video_icon"></span></p></dt><dd class="info rel"><p class="title" id="rentid_D09_1_02"><a href="/chuzu/3_205056212_1.htm" data_channel="1,2" source_page="1" target="_blank" title="两室两卫,拎包入住,轻轨站旁,随时看房!

In [4]:
# 初始化存储数据的列表
all_data = []

# 遍历每个HTML字符串
for html in string_list:
    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # 找到所有的 <dl> 标签
    for dl in soup.find_all('dl', class_='list hiddenMap rel'):
        # 提取标题
        title = dl.find('p', class_='title').find('a').get_text(strip=True)
        
        # 提取房屋信息
        house_info = dl.find('p', class_='font15 mt12 bold').get_text(strip=True)
        house_info_parts = house_info.split('|')
        house_info_parts = [part.strip() for part in house_info_parts]  # 去除多余空格
        
        # 提取位置信息
        location = dl.find('p', class_='gray6 mt12').get_text(strip=True)
        
        # 提取价格信息
        price = dl.find('span', class_='price').get_text(strip=True)
        
        # 初始化各部分的变量
        house_type = area = orientation = None
        
        # 根据分割后的部分数量，逐个赋值
        if len(house_info_parts) >= 1:
            house_type = house_info_parts[1]
        if len(house_info_parts) >= 2:
            area = house_info_parts[2]
        if len(house_info_parts) >= 3:
            orientation = house_info_parts[3]
        
        # 将提取的信息存储为字典
        data = {
            '标题': title,
            '户型': house_type,
            '面积（㎡）': area,
            '朝向': orientation,
            '位置': location,
            '价格（元/月）': price
        }
        # 将字典添加到列表中
        all_data.append(data)

# 将列表转换为DataFrame
df = pd.DataFrame(all_data)
df['block'] = '空港'  # 添加区域列


In [5]:
df

,标题,户型,面积（㎡）,朝向,位置,价格（元/月）,block
0,"两室两卫,拎包入住,轻轨站旁,随时看房!",2室2厅,37㎡,朝南,渝北-空港-桥达茂宸广场,1300,空港
1,空港广场 江北机场 三号线轻轨 房东直租 押一付一!,2户合租,23㎡,朝南,渝北-空港-远洋航港,600,空港
2,空港广场 三号线!江北机场!华夏航空!押一付一!!,2户合租,21㎡,朝南,渝北-空港-瑞丰花苑,600,空港
3,渝北空港圣地 3室2厅 130.67㎡ 1600元/,3室2厅,130㎡,朝北,渝北-空港-空港圣地,1600,空港
4,渝北金科空港城 3室1厅 82㎡ 1700元/月,3室1厅,82㎡,朝南北,渝北-空港-金科空港城,1700,空港
...,...,...,...,...,...,...,...
495,整租·红树林3室2厅,3室2厅,106㎡,朝南北,渝北-空港-红树林小区,1600,空港
496,整租·长安锦尚城 3室2厅 东南/南,3室2厅,85㎡,朝东南,渝北-空港-长安锦尚城,1600,空港
497,整租·瑞丰花苑2室1厅,2室1厅,80㎡,朝西,渝北-空港-瑞丰花苑,1600,空港
498,整租·环湖雅居B区B组团 2室1厅 南,2室1厅,68㎡,朝南,渝北-空港-环湖雅居,1300,空港


In [6]:
driver.quit()

In [7]:
def m_squre_to_number(s):
	if isinstance(s, str) and '㎡' in s:
		return float(s.replace('㎡', '')) 
	try:
		return float(s)
	except ValueError:
		return pd.NA

In [8]:
df['面积（㎡）'] = df['面积（㎡）'].apply(m_squre_to_number)

In [9]:
df.to_parquet("konggang_housing_rent.parquet", index=False)

In [10]:
df.to_csv('konggang_housing_rent.csv', index=False, encoding='utf-8-sig')